# 교차검증
- Train(test데이터는 제쳐두고) 를 여러번 나눠서 검증

In [1]:
import numpy as np
from sklearn.model_selection import KFold

In [2]:
X = np.array([
    [1,2],[3,4],[1,2],[3,4]
])

Y = np.array([1,2,3,4
])

In [3]:
X

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

In [4]:
Y

array([1, 2, 3, 4])

In [7]:
kf = KFold(n_splits=2)

print(kf.get_n_splits(X))

2


In [11]:
for train_idx, test_idx in kf.split(X):
    print('---idx')
    print(train_idx,test_idx)
    print('---train data')
    print(X[train_idx])
    print('---test data')
    print(X[test_idx])

---idx
[2 3] [0 1]
---train data
[[1 2]
 [3 4]]
---test data
[[1 2]
 [3 4]]
---idx
[0 1] [2 3]
---train data
[[1 2]
 [3 4]]
---test data
[[1 2]
 [3 4]]


In [18]:
import pandas as pd
red_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/winequality-red.csv'
white_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/winequality-white.csv'

red_wine = pd.read_csv(red_url,sep=';')
white_wine = pd.read_csv(white_url,sep=';')

red_wine['color'] = 1
white_wine['color'] = 0

wine = pd.concat([red_wine,white_wine])

In [19]:
wine['taste'] = [1 if grade > 5 else 0 for grade in wine['quality']]

X = wine.drop(['taste','quality'], axis=1)
Y = wine['taste']

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=13)

wine_tree_ss = DecisionTreeClassifier(max_depth=2, random_state=13)
wine_tree_ss.fit(X_train,y_train)

y_pred_tr = wine_tree_ss.predict(X_train) #훈련데이터에 대한 정확도검증
y_pred_test = wine_tree_ss.predict(X_test)

print('Train Acc:',accuracy_score(y_train,y_pred_tr))
print('Test Acc:',accuracy_score(y_test,y_pred_test))

Train Acc: 0.7294593034442948
Test Acc: 0.7161538461538461


In [20]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5)
wine_tree_cv = DecisionTreeClassifier(max_depth=2,random_state=13)

Kfold는 index를 반환한다.

In [22]:
for train_idx,test_idx in kfold.split(X):
    print(len(train_idx),len(test_idx))

5197 1300
5197 1300
5198 1299
5198 1299
5198 1299


그럼 학습은 어떻게?
- 각각 fold에 대한 학습 후 acc

In [25]:
cv_accuracy = []

for train_idx,test_idx in kfold.split(X):
    x_train = X.iloc[train_idx]
    x_test = X.iloc[test_idx]
    y_train = Y.iloc[train_idx]
    y_test = Y.iloc[test_idx]

    wine_tree_cv.fit(x_train,y_train)
    predict = wine_tree_cv.predict(x_test)
    cv_accuracy.append(accuracy_score(y_test,predict))


cv_accuracy

[0.6007692307692307,
 0.6884615384615385,
 0.7090069284064665,
 0.7628945342571208,
 0.7867590454195535]

각 acc의 분산이 크지 않다면 kfold의 평균값으로 구한다

In [26]:
np.mean(cv_accuracy)

0.709578255462782

In [28]:
from sklearn.model_selection import StratifiedKFold

stkfold = StratifiedKFold(n_splits=5)
wine_tree_cv = DecisionTreeClassifier(max_depth=2,random_state=13)

cv_accuracy = []

for train_idx,test_idx in stkfold.split(X,Y):
    x_train = X.iloc[train_idx]
    x_test = X.iloc[test_idx]
    y_train = Y.iloc[train_idx]
    y_test = Y.iloc[test_idx]

    wine_tree_cv.fit(x_train,y_train)
    predict = wine_tree_cv.predict(x_test)
    cv_accuracy.append(accuracy_score(y_test,predict))


cv_accuracy

[0.5523076923076923,
 0.6884615384615385,
 0.7143956889915319,
 0.7321016166281755,
 0.7567359507313318]

In [29]:
np.mean(cv_accuracy)

0.6888004974240539

Cross validation을 간편하게
- cross_val_score

In [31]:
from sklearn.model_selection import cross_val_score

stkfold = StratifiedKFold(n_splits=5)
wine_tree_cv = DecisionTreeClassifier(max_depth=2,random_state=13)

cross_val_score(wine_tree_cv,X,Y,cv=stkfold)

array([0.55230769, 0.68846154, 0.71439569, 0.73210162, 0.75673595])

In [32]:
#max_depth 바꿔보기
from sklearn.model_selection import cross_val_score

stkfold = StratifiedKFold(n_splits=5)
wine_tree_cv = DecisionTreeClassifier(max_depth=5,random_state=13)

cross_val_score(wine_tree_cv,X,Y,cv=stkfold)

array([0.50076923, 0.62615385, 0.69745958, 0.7582756 , 0.74903772])

In [36]:
def skfold_dt(depth):
    from sklearn.model_selection import cross_val_score

    stkfold = StratifiedKFold(n_splits=5)
    wine_tree_cv = DecisionTreeClassifier(max_depth=depth,random_state=13)

    return cross_val_score(wine_tree_cv,X,Y,cv=stkfold)

In [37]:
skfold_dt(3)

array([0.56846154, 0.68846154, 0.71439569, 0.73210162, 0.75673595])

만약 train score를 함께보고싶다면?
- cross_validation

In [41]:
from sklearn.model_selection import cross_validate

stkfold = StratifiedKFold(n_splits=5)
wine_tree_cv = DecisionTreeClassifier(max_depth=5,random_state=13)

cross_validate(wine_tree_cv,X,Y,cv=stkfold,return_train_score=True)

{'fit_time': array([0.01906204, 0.0210278 , 0.02005816, 0.02023482, 0.02003741]),
 'score_time': array([0.00300503, 0.00199986, 0.00250888, 0.00251245, 0.00251365]),
 'test_score': array([0.50076923, 0.62615385, 0.69745958, 0.7582756 , 0.74903772]),
 'train_score': array([0.78795459, 0.78045026, 0.77568295, 0.76356291, 0.76279338])}

현재 과적합이 있는것을 확인할 수 있다.

----

# 하이퍼파라미터 튜닝
- 모델의 성능을 확보하기 위해 조절하는 설정 값

In [42]:
import pandas as pd
red_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/winequality-red.csv'
white_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/winequality-white.csv'

red_wine = pd.read_csv(red_url,sep=';')
white_wine = pd.read_csv(white_url,sep=';')

red_wine['color'] = 1
white_wine['color'] = 0

wine = pd.concat([red_wine,white_wine])

wine['taste'] = [1 if grade > 5 else 0 for grade in wine['quality']]

X = wine.drop(['taste','quality'], axis=1)
Y = wine['taste']

#### GridSearchCV

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'max_depth' : [2,4,7,10]}
wine_tree = DecisionTreeClassifier(max_depth=2, random_state=13)

In [44]:
gridsearch = GridSearchCV(estimator=wine_tree, param_grid=params, cv=5)
gridsearch.fit(X,Y)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=2, random_state=13),
             param_grid={'max_depth': [2, 4, 7, 10]})

In [45]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(gridsearch.cv_results_)

{   'mean_fit_time': array([0.00985618, 0.01774588, 0.02606993, 0.04822617]),
    'mean_score_time': array([0.00262771, 0.00217185, 0.00412345, 0.00220222]),
    'mean_test_score': array([0.6888005 , 0.66356523, 0.65340854, 0.64401587]),
    'param_max_depth': masked_array(data=[2, 4, 7, 10],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object),
    'params': [   {'max_depth': 2},
                  {'max_depth': 4},
                  {'max_depth': 7},
                  {'max_depth': 10}],
    'rank_test_score': array([1, 2, 3, 4]),
    'split0_test_score': array([0.55230769, 0.51230769, 0.50846154, 0.51615385]),
    'split1_test_score': array([0.68846154, 0.63153846, 0.60307692, 0.60076923]),
    'split2_test_score': array([0.71439569, 0.72363356, 0.68360277, 0.66743649]),
    'split3_test_score': array([0.73210162, 0.73210162, 0.73672055, 0.71054657]),
    'split4_test_score': array([0.75673595, 0.7182448 , 0.73518091, 0.72517321]),
    'std

In [46]:
gridsearch.best_estimator_

DecisionTreeClassifier(max_depth=2, random_state=13)

In [47]:
gridsearch.best_score_

0.6888004974240539

In [48]:
gridsearch.best_params_

{'max_depth': 2}

만약 Pipeline에 GridSearchCV를 적용하고 싶다면?

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

estimators = [
    ('scaler',StandardScaler()),
    ('clf', DecisionTreeClassifier())
]

pipe = Pipeline(estimators)

In [50]:
param_grid = [{"clf__max_depth" : [2,4,7,10]}]

GridSearch= GridSearchCV(estimator=pipe, param_grid=param_grid,cv=5)
GridSearch.fit(X,Y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf', DecisionTreeClassifier())]),
             param_grid=[{'clf__max_depth': [2, 4, 7, 10]}])

In [51]:
GridSearch.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', DecisionTreeClassifier(max_depth=2))])

In [52]:
GridSearch.best_score_

0.6888004974240539

In [53]:
score_df = pd.DataFrame(GridSearch.cv_results_)
score_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018589,0.014599,0.003314,0.000691,2,{'clf__max_depth': 2},0.552308,0.688462,0.714396,0.732102,0.756736,0.688800,0.071799,1
1,0.023125,0.007119,0.002726,0.000534,4,{'clf__max_depth': 4},0.512308,0.631538,0.723634,0.732102,0.718245,0.663565,0.083905,2
2,0.025472,0.001466,0.002277,0.000499,7,{'clf__max_depth': 7},0.519231,0.605385,0.677444,0.736721,0.735181,0.654792,0.083073,3
3,0.038031,0.001472,0.001729,0.000582,10,{'clf__max_depth': 10},0.518462,0.612308,0.667436,0.712086,0.722094,0.646477,0.074856,4


표로 성능결과 정리하기

In [54]:
score_df[['params','rank_test_score','mean_test_score','std_test_score']]

,params,rank_test_score,mean_test_score,std_test_score
0,{'clf__max_depth': 2},1,0.688800,0.071799
1,{'clf__max_depth': 4},2,0.663565,0.083905
2,{'clf__max_depth': 7},3,0.654792,0.083073
3,{'clf__max_depth': 10},4,0.646477,0.074856
